In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import warnings
warnings.filterwarnings("ignore")


In [26]:
url_list = []
for i in range(1, 42):
    url = f"https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree&page={i}"
    url_list.append(url)

In [27]:
def create_html_page(url):
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page_html, features='html')
    return page_soup

In [28]:
def create_final_data(page_soup):
    data={'Product Name': [],
          'Sale & Actual Price': [],
          'Raview & Rattings': [],
          'Offer': [],
          'Camera': [],
          'Memory': [],
          'Battery': [],
          'Display': [],
          'Ratting Number': []}
    
    all_info_class_name= "ZFwe0M row"
    containers = page_soup.find_all("div", {"class": all_info_class_name})
    
    for contain in containers:
        product_name = contain.find_all("div", {"class": "RG5Slk"})[0].text
        data["Product Name"].append(product_name)
    
        rr = contain.find_all("span", {"class": "PvbNMB"})
        if len(rr) == 0:
           review_and_ratting = None
           ratting_number = None
        else:
           review_and_ratting = rr[0].text
           ratting_number = contain.find_all("span", {"class": "CjyrHS"})[0].text
        
        data["Raview & Rattings"].append(review_and_ratting)
        data["Ratting Number"].append(ratting_number)
        specifictations = contain.find_all("div", {"class": "CMXw7N"})[0]
        data["Memory"].append(specifictations.find_all('li')[0].text)
        data["Display"].append(specifictations.find_all('li')[1].text)
        data["Camera"].append(specifictations.find_all('li')[2].text)
        data["Battery"].append(specifictations.find_all('li')[3].text)
    
        price = contain.find_all("div", {"class": "QiMO5r"})[0].text
        data["Sale & Actual Price"].append(price)
    
        offer_list = contain.find_all("div", {"class": "HQe8jr"})
        if len(offer_list) == 0:
           data["Offer"].append("No Offer")
        else:
           data["Offer"].append(offer_list[0].text)
    return data

In [ ]:
data_frame_list = []
for i, url in enumerate(url_list):
    print(f"Started url page {i+1}")
    page_soup = create_html_page(url)
    data_info = create_final_data(page_soup)
    data_frame_list.append(data_info)

In [43]:
len(data_frame_list)

25

In [36]:
df_all = []
for i in range(len(data_frame_list)):
    df = pd.DataFrame(data_frame_list[i])
    df_all.append(df)

In [37]:
final_df = pd.concat(df_all, ignore_index=True)

In [38]:
final_df["Product Name"].unique().shape

(267,)

In [39]:
final_df.shape

(600, 9)

In [40]:
final_df.to_csv("flipkart_mobile_phones.csv")